<a href="https://colab.research.google.com/github/talpt/pyton/blob/main/Artan_Fiyat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed matplotlib openpyxl tqdm
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
import warnings
from tradingview_screener import get_all_symbols
from tqdm import tqdm  # İlerleme çubuğu için gerekli kütüphane

warnings.simplefilter(action='ignore')

# TradingView'e giriş yapmadan bağlanın
tv = TvDatafeed()

# Türkiye piyasasındaki tüm hisse senetlerini otomatik olarak alın
symbols = get_all_symbols(market='turkey')
symbols = [symbol.replace('BIST:', '') for symbol in symbols]
symbols = sorted(symbols)

# Verileri saklamak için boş bir DataFrame oluşturun
filtered_data = []

# İlerleme çubuğu ile her bir hisse senedi için verileri alın
for symbol in tqdm(symbols, desc="İşlem yapılıyor", unit="symbol"):
    try:
        # Aylık veri çekin (örnek olarak son 2 ay)
        df_monthly = tv.get_hist(symbol=symbol, exchange='BIST', interval=Interval.in_monthly, n_bars=2)

        # Verinin yeterli olup olmadığını kontrol edin
        if df_monthly is None or len(df_monthly) < 2:
            print(f"{symbol} için yeterli aylık veri yok, atlanıyor.")
            continue

        # Haftalık veri çekin (örnek olarak son 2 hafta)
        df_weekly = tv.get_hist(symbol=symbol, exchange='BIST', interval=Interval.in_weekly, n_bars=2)

        # Verinin yeterli olup olmadığını kontrol edin
        if df_weekly is None or len(df_weekly) < 2:
            print(f"{symbol} için yeterli haftalık veri yok, atlanıyor.")
            continue

        # Günlük veriyi alın (bugünkü kapanış fiyatı için son 1 gün)
        df_daily = tv.get_hist(symbol=symbol, exchange='BIST', interval=Interval.in_daily, n_bars=1)

        # Günlük verinin yeterli olup olmadığını kontrol edin
        if df_daily is None or len(df_daily) < 1:
            print(f"{symbol} için güncel fiyat verisi yok, atlanıyor.")
            continue

        # Kapanış fiyatlarını al
        monthly_close_prev = df_monthly['close'].iloc[-2]  # Önceki ay kapanışı
        weekly_close_prev = df_weekly['close'].iloc[-2]  # Önceki hafta kapanışı
        current_price = df_daily['close'].iloc[-1]  # Güncel kapanış fiyatı

        # Filtreleme kriteri: Güncel fiyat hem bir önceki ayın hem de bir önceki haftanın kapanışının üzerinde olmalı
        if current_price > monthly_close_prev and current_price > weekly_close_prev:
            filtered_data.append([symbol, current_price, monthly_close_prev, weekly_close_prev])

    except Exception as e:
        print(f"{symbol} işlenirken hata oluştu: {e}")

# Sonuçları DataFrame'e dönüştürün
df_filtered_results = pd.DataFrame(filtered_data, columns=['Symbol', 'Current Price', 'Previous Month Close', 'Previous Week Close'])

# Sonuçları ekrana yazdırın
print("Bir önceki ayın ve haftanın kapanış fiyatlarının üzerinde olan hisse senetleri:")
display(df_filtered_results)

# Verileri Excel dosyasına kaydedin
df_filtered_results.to_excel("filtered_above_previous_month_week.xlsx", index=False)
print("Filtrelenen veriler başarıyla Excel dosyasına kaydedildi.")



  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-gb3v2fo5
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-gb3v2fo5
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=0d3f800b6775ae2181ed4c2aa4574a6e6e4af11a274a508be4c3b783dc0f5257
  Stored in directory: /tmp/pip-ephem-wheel-cache-ams3xx34/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.5 MB/s eta 0:00:00


İşlem yapılıyor:   8%|▊         | 49/585 [01:37<12:57,  1.45s/symbol]

ARMGD için yeterli aylık veri yok, atlanıyor.


İşlem yapılıyor:  30%|██▉       | 175/585 [05:43<09:07,  1.34s/symbol]

EGEGY için yeterli aylık veri yok, atlanıyor.


...

GUNDG için güncel fiyat verisi yok, atlanıyor.


İşlem yapılıyor: 100%|██████████| 585/585 [18:24<00:00,  1.89s/symbol]

Bir önceki ayın ve haftanın kapanış fiyatlarının üzerinde olan hisse senetleri:


,Symbol,Current Price,Previous Month Close,Previous Week Close
0,AGROT,10.89,10.68,10.34
1,AHSGY,24.88,21.70,23.56
2,AKBNK,68.55,64.85,66.00
3,AKCNS,188.00,180.00,184.60
4,ALBRK,7.35,6.43,7.28
...,...,...,...,...
131,VRGYO,3.27,3.00,3.11
132,YAPRK,888.00,650.50,860.00
133,YGGYO,66.35,62.30,65.05
134,YKBNK,32.58,30.62,31.70


Filtrelenen veriler başarıyla Excel dosyasına kaydedildi.
